## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [1]:
#pull the ara results and look through KF+G edge graphs and pull the povenance attribute and make a table that will look like
#rows will be kp
#col be workflows
#entry would be the ara's that had edges in kp in the workflow. 

In [2]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd


#https://ars.ci.transltr.io/ars/api --- 
#https://arax.ci.transltr.io -- backup url

In [264]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [317]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

### Submiting more than one jobs (test)

<br>

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

<br>

## Test for individual run

<br>

In [265]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.6_metformin-ferritin.json') as inf:
    query1 = json.load(inf)

In [266]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": [
                        "CHEMBL.COMPOUND:CHEMBL1431"
                    ],
                    "categories": [
                        "biolink:SmallMolecule"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:Protein"
                    ]
                },
                "n2": {
                    "categories": [
                        "biolink:Protein"
                    ]
                },
                "n3": {
                    "ids": [
                        "UniProtKB:P02794",
                        "UniProtKB:P02792"
                    ],
                    "categories": [
                        "biolink:Protein"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
             

In [267]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=ca41b368-c9e3-433d-94e2-14f12d3e0a4a


In [318]:
retrieve_ars_results(kcresult)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [316]:
kcresult = 'https://arax.ncats.io/?source=ARS&id=ca41b368-c9e3-433d-94e2-14f12d3e0a4a'
ars_url='https://ars.transltr.io/ars/api'
pk = 'https://arax.ci.transltr.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [304]:
#message_url = f'{ars_url}/messages/{kcresult}?trace=y'

In [305]:
message_url

'https://ars.transltr.io/ars/api/messages/https://arax.ci.transltr.io/?source=ARS&id=3968469b-80f0-4da9-b37a-12713fa1a5c3?trace=y'

In [306]:
response = requests.get(message_url)
j = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [108]:
j.keys()

dict_keys(['message', 'status', 'actor', 'children'])

In [192]:
j

{'message': '092daa37-6612-418d-830a-b3cd8c727c5f',
 'status': 'Done',
 'actor': {'pk': 9,
  'channel': 'general',
  'agent': 'ars-default-agent',
  'path': ''},
 'children': [{'message': '4547bbf0-a425-4495-b587-1aa0cec4640e',
   'status': 'Done',
   'code': 200,
   'actor': {'pk': 1,
    'channel': 'general',
    'agent': 'ara-aragorn',
    'path': 'runquery'},
   'children': []},
  {'message': '8e65b2b3-2713-46d6-8b71-9e44f322e260',
   'status': 'Done',
   'code': 200,
   'actor': {'pk': 2,
    'channel': 'general',
    'agent': 'ara-arax',
    'path': 'runquery'},
   'children': []},
  {'message': '54b52c25-6089-4f89-b836-6ce9e67f5b30',
   'status': 'Done',
   'code': 200,
   'actor': {'pk': 3,
    'channel': 'general',
    'agent': 'ara-bte',
    'path': 'runquery'},
   'children': []},
  {'message': '8e7c070a-87d9-4d0e-adb2-eb0f8a4f76f5',
   'status': 'Done',
   'code': 200,
   'actor': {'pk': 6,
    'channel': 'general',
    'agent': 'ara-unsecret',
    'path': 'runquery'},
   '

{'message': 'abf7c1e8-29bd-451b-a681-4f3fb42d1ee3', 'status': 'Error', 'code': 422, 'actor': {'pk': 1, 'channel': 'general', 'agent': 'ara-aragorn', 'path': 'runquery'}, 'children': []}
{'message': 'e78e1d63-0f77-47f4-9f7a-171e4fe874dd', 'status': 'Error', 'code': 400, 'actor': {'pk': 2, 'channel': 'general', 'agent': 'ara-arax', 'path': 'runquery'}, 'children': []}
{'message': '1c3f0b00-a5b4-4189-a41a-56d82fc53dd9', 'status': 'Done', 'code': 200, 'actor': {'pk': 3, 'channel': 'general', 'agent': 'ara-bte', 'path': 'runquery'}, 'children': []}
{'message': '158bcb1a-22bc-4c88-b893-a79d3ee7ba88', 'status': 'Done', 'code': 200, 'actor': {'pk': 6, 'channel': 'general', 'agent': 'ara-unsecret', 'path': 'runquery'}, 'children': []}
{'message': '0194c27e-afe8-4506-955e-b38cae28cc37', 'status': 'Error', 'code': 400, 'actor': {'pk': 7, 'channel': 'general', 'agent': 'kp-genetics', 'path': 'runquery'}, 'children': []}
{'message': 'df65c159-bf39-4867-9a2e-a530936f4e2b', 'status': 'Error', 'code':

In [255]:
dictionary_test = {}
for child in j['children']:
    #print(child['status'])
    dictionary_test
    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                #print(val)
                test_att_values =[]
                for tx in edge_ex[val]['attributes']:
                    print(tx)
                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
                        #print(tx['value'])
                        test_att_values.append(tx['value'])
                        dictionary_test[child['actor']['agent']] = test_att_values
        else:
            pass
    

{'attribute_type_id': 'biolink:primary_knowledge_source', 'value': ['infores:semmeddb'], 'value_type_id': 'biolink:InformationResource'}
{'attribute_type_id': 'biolink:aggregator_knowledge_source', 'value': ['SEMMED Gene API'], 'value_type_id': 'biolink:InformationResource'}
{'attribute_type_id': 'publications', 'value': ['PMID:24356943'], 'value_type_id': 'biolink:publication'}
{'attribute_source': 'infores:arax', 'attribute_type_id': 'biolink:knowledge_source', 'description': 'ARAX inserted this attribute because the KP (BTE) did not seem to provide such an attribute (indicating that this edge came from them).', 'value': 'infores:biothings-explorer', 'value_type_id': 'biolink:InformationResource'}
{'attribute_source': 'infores:arax', 'attribute_type_id': 'biolink:aggregator_knowledge_source', 'value': 'infores:arax', 'value_type_id': 'biolink:InformationResource'}
{'attribute_type_id': 'biolink:primary_knowledge_source', 'value': ['infores:semmeddb'], 'value_type_id': 'biolink:Inform

{'attribute_source': 'infores:spoke', 'attribute_type_id': 'biolink:original_knowledge_source', 'original_attribute_name': 'source', 'value': 'infores:lincs', 'value_type_id': 'biolink:InformationResource'}
{'attribute_source': 'infores:spoke', 'attribute_type_id': 'biolink:aggregator_knowledge_source', 'value': 'infores:spoke', 'value_type_id': 'biolink:InformationResource'}
{'attribute_source': 'infores:improving-agent', 'attribute_type_id': 'biolink:aggregator_knowledge_source', 'value': 'infores:improving-agent', 'value_type_id': 'biolink:InformationResource'}
{'attribute_source': 'infores:lincs', 'attribute_type_id': 'biolink:has_evidence', 'original_attribute_name': 'zscore', 'value': -6.798004208569094}
{'attribute_source': 'infores:lincs', 'attribute_type_id': 'biolink:p_value', 'original_attribute_name': 'pvalue', 'value': 9.754965766450568e-08}
{'attribute_source': 'infores:spoke', 'attribute_type_id': 'biolink:original_knowledge_source', 'original_attribute_name': 'source', 

In [254]:
if tx['attribute_type_id'] == 'biolink:primary_knowledge_source':
    print(tx['value'])
else:
    print('Nothing')

Nothing


In [256]:
dictionary_test

{'ara-arax': ['infores:rtx-kg2', 'infores:semmeddb', 'infores:arax'],
 'ara-bte': [['infores:semmeddb'], ['SEMMED Gene API']],
 'ara-improving': ['infores:lincs',
  'infores:spoke',
  'infores:improving-agent']}

In [257]:
dictionary={}
results = {}
dictionary_2 = {}
for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                print(val)
                                test_att_values =[]
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
                                        test_att_values.append(tx['value'])
                                        dictionary_2[child['actor']['agent']] = test_att_values
                    else:
                        pass 
                else:
                    dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)

ARS Error
ara-aragorn ARS Error 0
Done
BTE:NCBIGene:100616101-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:1869-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:5241-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:5555-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:7332-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:UMLS:C0079427-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:UMLS:C1418864-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:UMLS:C4085573-biolink:entity_negatively_regulates_entity-NCBIGene:23221
N1_0
N1_1
N1_2
N1_3
N1_4
N1_5
N1_6
N1_7
N1_8
RTX-KG2:UniProtKB:Q01094-biolink:entity_negatively_regulates_entity-UniProtKB:Q9BYZ6
RTX-KG2:UniProtKB:Q13618-biolink:entity_negatively_regulates_entity-UniProtKB:Q9BYZ6
ara-arax Done 9
Done
NCBIGene:100616101-biolink:entity_negatively_regulates_entity-NCBIGene:23221
UMLS:C4085573-biolink:entity_negati

In [258]:
dictionary_2

{'ara-aragorn': [],
 'ara-arax': ['infores:rtx-kg2', 'infores:semmeddb', 'infores:arax'],
 'ara-bte': [['infores:semmeddb'], ['SEMMED Gene API']],
 'ara-unsecret': [],
 'kp-genetics': [],
 'kp-molecular': [],
 'ara-explanatory': [],
 'ara-improving': ['infores:lincs',
  'infores:spoke',
  'infores:improving-agent'],
 'kp-cam': [],
 'kp-textmining': [],
 'ara-aragorn-exp': [],
 'kp-openpredict': [],
 'kp-icees': [],
 'ara-robokop': [],
 'kp-chp': [],
 'kp-cohd': [],
 'kp-icees-dili': []}

In [231]:
for child in j['children']:
    #print(child['status'])
    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        if child_response['fields']['data']['message']['knowledge_graph']['edges']:
            print('correct')
        else:
            print('incorrect')

correct
correct
incorrect
incorrect
incorrect
incorrect
correct
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect


In [167]:
child_response['fields']['data']['message']['knowledge_graph']

{'nodes': {}, 'edges': {}}

In [126]:
mm = child_response['pk']
mm

'615f54a8-cb94-497f-9152-ec883597c17b'

In [128]:
xt = f'{ars_url}/messages/{childmessage_id}/{mm}'
xt

'https://ars.transltr.io/ars/api/messages/615f54a8-cb94-497f-9152-ec883597c17b/615f54a8-cb94-497f-9152-ec883597c17b'

In [94]:
f'{ars_url}/model/{mm}'

'https://ars.transltr.io/ars/api/model/615f54a8-cb94-497f-9152-ec883597c17b'

In [98]:
requests.get(xt).json

<bound method Response.json of <Response [200]>>

In [ ]:
childmessage_id = child['message']
child_url = f'{ars_url}/messages/{childmessage_id}'

 child_response = requests.get(child_url).json()

In [ ]:
result_status = retrieve_ars_results(kcresult)

In [ ]:
kcresult

In [ ]:
result_status 

In [ ]:
pwd

In [ ]:
df

In [ ]:
def make_hyperlink(value):
    #url = "https://custom.url/{}"
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [ ]:
make_hyperlink('https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482')

In [ ]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [ ]:
df

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
df

In [244]:
def test_return(a,b):
    y = a+b
    x = a-b
    
    return [y,x]

In [245]:
test_return(5,10)

[15, -5]